# Semi-structured eval: baseline retriever

We will test retrival of table information from the `Semi-structured Reports` dataset using various methods.

## Pre-requisites

In [ ]:
%pip install -U langchain langsmith langchain_benchmarks
%pip install --quiet chromadb openai

## Dataset

In [11]:
import os
from pathlib import Path

from langchain_benchmarks import clone_public_dataset, registry
from langchain_benchmarks.rag.tasks.semi_structured_reports import get_file_names

# Task
task = registry["Semi-structured Reports"]

# Files used
paths = list(get_file_names())
files = [str(p) for p in paths]

### TODO: Replace when dataset is updated
dir = "/Users/rlm/Desktop/Eval_Sets/semi_structured_reports/"
files = [f for f in os.listdir(dir) if f.endswith(".pdf")]

## Load

In [30]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


def load_and_split(file):
    """
    Load and split PDF files
    """

    loader = PyPDFLoader(file)
    pdf_pages = loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=100, chunk_overlap=50
    )

    # Get chunks
    docs = text_splitter.split_documents(pdf_pages)
    texts = [d.page_content for d in docs]
    print(f"There are {len(texts)} text elements")
    return texts


texts = []
for fi in files:
    texts.extend(load_and_split(dir + fi))

There are 59 text elements
There are 77 text elements
There are 67 text elements
There are 285 text elements
There are 146 text elements
There are 130 text elements


## Index

In [31]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore_baseline = Chroma.from_texts(
    texts=texts, collection_name="baseline-100-token", embedding=OpenAIEmbeddings()
)

retriever_baseline = vectorstore_baseline.as_retriever()

## RAG

In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough


def rag_chain(retriever):
    """
    RAG chain
    """

    # Prompt template
    template = """Answer the question based only on the following context, which can include text and tables:
    {context}
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    # LLM
    model = ChatOpenAI(temperature=0, model="gpt-4")

    # RAG pipeline
    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )
    return chain


# Create RAG chain
chain = rag_chain(retriever_baseline)

## Eval

In [ ]:
### TODO: Replace with public dataset

import uuid

import pandas as pd
from langsmith import Client

# Read
df = pd.read_csv(dir + "semi_structured_reports.csv")

# Dataset
client = Client()
dataset_name = f"Semi-Structured-Eval"
dataset = client.create_dataset(dataset_name=dataset_name)

# Populate dataset
for _, row in df.iterrows():
    # Get Q, A
    q = row["Question"]
    a = row["Answer"]

    # Use the values in your function
    client.create_example(
        inputs={"question": q}, outputs={"answer": a}, dataset_id=dataset.id
    )

In [33]:
from functools import partial

from langsmith.client import Client

from langchain_benchmarks.rag import get_eval_config


def run_eval(chain, eval_run_name):
    """
    Run eval
    """
    client = Client()
    test_run = client.run_on_dataset(
        ### TODO: Replace with public dataset
        dataset_name="Semi-Structured-Eval",
        llm_or_chain_factory=lambda: (lambda x: x["question"]) | chain,
        evaluation=get_eval_config(),
        verbose=True,
        project_name=eval_run_name,
    )


# Experiments
chain_map = {
    "baseline-100tok": chain,
}

run_id = str(uuid.uuid4())
for project_name, chain in chain_map.items():
    run_eval(chain, project_name + "_" + run_id)

View the evaluation results for project 'baseline-100tok_50badcda-71d1-4240-bd84-99119459e0be' at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/projects/p/959d8014-07d9-4d5b-935a-72565baaca1a?eval=true

View all tests for Dataset Semi-Structured-Eval at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/datasets/95f61109-029d-43a9-ae7d-ec1d53c6f723
[------------------------------------------------->] 20/20
 Eval quantiles:
                               0.25       0.5      0.75      mean  \
embedding_cosine_distance  0.037430  0.086165  0.116285  0.076345   
score_string:accuracy      0.100000  0.100000  1.000000  0.465000   
faithfulness               0.875000  1.000000  1.000000  0.805000   
execution_time             7.181396  7.181396  7.181396  7.181396   

                                   mode  
embedding_cosine_distance  6.962568e-07  
score_string:accuracy      1.000000e-01  
faithfulness               1.000000e+00  
execution_time      